# Snake Multiplayer

In [25]:
%serialconnect %serialconnect --port=/dev/cu.usbserial-0001 --baud=115200 

serial exception on close write failed: [Errno 6] Device not configured
Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 
Ready.


In [26]:
import machine
import display
import random

from time import sleep

display = display.Display()

In [27]:
class Coordinate:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return f"({self.x}/{self.y})"


In [28]:
class Playground:
    DISPLAY_WIDTH = 128
    DISPLAY_HEIGHT = 64

    BLOCK_SIZE = 4

    GAME_WIDTH = int(DISPLAY_WIDTH / BLOCK_SIZE)
    GAME_HEIGHT = int(DISPLAY_HEIGHT / BLOCK_SIZE)

    def __init__(self):
        pass

    def draw_coordinate(self, coordintate: Coordinate):
        display.fill_rect(coordintate.x * self.BLOCK_SIZE, coordintate.y * self.BLOCK_SIZE, self.BLOCK_SIZE , self.BLOCK_SIZE)

In [29]:
class Apple:    
    def __init__(self, playground):
        self.playground = playground
        x = random.randint(0, self.playground.GAME_WIDTH - 1)
        y = random.randint(0, self.playground.GAME_HEIGHT - 1)
        self.coordinate = Coordinate(x, y)
    
    def draw_apple(self):
        self.playground.draw_coordinate(self.coordinate)

In [30]:
class Snake:
    START_SNAKE_LENGTH = 2

    def __init__(self, playground: Playground, startPos: Coordinate):
        self.tails = []
        self.length = self.START_SNAKE_LENGTH
        self.direction = 2 # 0 = up, 1 down, 2 left, 3 right
        self.score = 0
        self.tails.append(startPos)
        self.playground = playground


    def get_score(self) -> int:
        return self.score


    def change_direction(self, newDirection):
        if (self.direction == 0 and newDirection == 1):
            return
        elif (self.direction == 1 and newDirection == 0):
            return
        elif (self.direction == 2 and newDirection == 3):
            return
        elif (self.direction == 3 and newDirection == 2):
            return

        self.direction = newDirection

    def moove_snake(self):
        head = self.tails[-1]
        newPos = Coordinate(head.x, head.y)

        if (self.direction == 0):
            newPos.y = head.y - 1
            if newPos.y < 0:
                newPos.y = self.playground.GAME_HEIGHT - 1
        elif (self.direction == 1):
            newPos.y = head.y + 1
            if newPos.y >= self.playground.GAME_HEIGHT:
                newPos.y = 0
        elif (self.direction == 2):
            newPos.x = head.x - 1
            if newPos.x < 0:
                newPos.x = self.playground.GAME_WIDTH - 1
        elif (self.direction == 3):
            newPos.x = head.x + 1
            if newPos.x >= self.playground.GAME_WIDTH:
                newPos.x = 0

        if (len(self.tails) >= self.length):
            self.tails.pop(0)

        self.tails.append(newPos)

    def detect_apple_colision(self, apple: Apple) -> bool:
        collisionDetected = self.detect_colision(apple.coordinate)
        if collisionDetected:
            self.length += 2
            self.score += 1
    
        return collisionDetected
    
    def detect_game_over(self):
        head = self.tails[-1]
        
        for pos in self.tails[0:-1]:
            if pos.x == head.x and pos.y == head.y:
                for i in range(0, max(self.START_SNAKE_LENGTH, len(self.tails) - self.START_SNAKE_LENGTH)):
                    self.tails.pop()
                self.length = self.START_SNAKE_LENGTH
                self.score = 0


    def detect_colision(self, coordinate) -> bool:
        head = self.tails[-1]
        return head.x == coordinate.x and head.y == coordinate.y

    def draw_snake(self):
        for tail in self.tails:
            self.playground.draw_coordinate(tail)

In [31]:
import uasyncio as asyncio
import aioble
from bluetooth import UUID
import time

import random
import struct
import display
from time import sleep
import json

display = display.Display()



class BleService:

    ADVERTISING_INTERVAL_MILLISECONDS = 250_000
    GATT_APPEARANCE = 768

    NAME = "SnakeMultiplayer"

    SNAKE_SERVICE_UUID = UUID("ca07faee-7e95-4856-b44b-bbaef52ec7b4")
    APPLE_POSITION_CHARACTERISTIC_UUID = UUID("cec23f9f-cdc0-4577-8798-8dd4b01724d8")
    SNAKE1_POSITIONS_CHARACTERISTIC_UUID = UUID("e72c988f-7279-4b82-b808-42884a4ba48f")
    SNAKE2_POSITIONS_CHARACTERISTIC_UUID = UUID("1673be03-ef60-4fb9-869c-4513a8e212f1")
    SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID = UUID("1533130f-7251-4a87-a976-1b5bc0fae798")

    snake1_transmitted_length = 0
    snake2_transmitted_length = 0

    def __init__(self):
        self.connection = None
        self.device_connected = False

        self.gatt_snake_service = aioble.Service(self.SNAKE_SERVICE_UUID)
        self.snake1_positions_characteristic = aioble.Characteristic(
            self.gatt_snake_service,
            self.SNAKE1_POSITIONS_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        self.snake2_positions_characteristic = aioble.Characteristic(
            self.gatt_snake_service,
            self.SNAKE2_POSITIONS_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        self.snake2_directions_characteristic = aioble.Characteristic(
            self.gatt_snake_service,
            self.SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        self.apple_position_characteristic = aioble.Characteristic(
            self.gatt_snake_service,
            self.APPLE_POSITION_CHARACTERISTIC_UUID,
            read=True,
            notify=True)
        
        aioble.register_services(self.gatt_snake_service)

    def update_snake1_tails(self, length: int, tails: list):
        self._udpate_snake_tails(length, self.snake1_transmitted_length, tails, self.snake1_positions_characteristic)
        self.snake1_transmitted_length = len(tails)
    
    def update_snake2_tails(self, length: int, tails: list):
        self._udpate_snake_tails(length, self.snake2_transmitted_length, tails, self.snake2_positions_characteristic)
        self.snake2_transmitted_length = len(tails)
    
    def update_apple_position(self, coordinate: Coordinate):
        self.apple_position_characteristic.write(f"[{coordinate.x}, {coordinate.y}]")

    def _udpate_snake_tails(self, length: int, transmitted_length: int, tails: list, characteristic):
        tailsToTransmitt = tails[max(0, transmitted_length - 1):]
        parsedTails = list(map(lambda d: (d.x, d.y), tailsToTransmitt))
        sendData = [length, parsedTails]
        sendData = str(json.dumps([length, parsedTails])).replace(" ", "")
        characteristic.write(sendData)


    async def advertise(self):
        if self.connection is not None:
            self.device_connected = self.connection.is_connected()

        if self.device_connected is False:
            self.connection = await aioble.advertise(
                self.ADVERTISING_INTERVAL_MILLISECONDS,
                name=self.NAME,
                services=[self.SNAKE_SERVICE_UUID],
                appearance=self.GATT_APPEARANCE,
            )
            self.device_connected = True

In [32]:
playground = Playground()

snakes = [
    Snake(playground, Coordinate(10, 10)),
    Snake(playground, Coordinate(20, 10))
]

apple = Apple(playground)

downBtn = Pin(14, Pin.IN, Pin.PULL_UP)
rightBtn = Pin(21, Pin.IN, Pin.PULL_UP)
leftBtn = Pin(18, Pin.IN, Pin.PULL_UP)
upBtn = Pin(25, Pin.IN, Pin.PULL_UP)

bleService = BleService()

def draw_score(score):
    display.text("Score " + str(score), 0, 0)

async def main():
    global bleService, display, downBtn, rightBtn, leftBtn, upBtn, apple, snakes, playground, draw_score

    mySnake = snakes[0]

    while True:
        display.clear()
        if bleService.device_connected is False:
            display.text("Waiting for", 0, 0)
            display.text("connection", 0, 20)
            display.show()

        await bleService.advertise()

        if downBtn.value() == 0:
            mySnake.change_direction(1)
        if rightBtn.value() == 0:
            mySnake.change_direction(3)
        if leftBtn.value() == 0:
            mySnake.change_direction(2)
        if upBtn.value() == 0:
            mySnake.change_direction(0)

        apple.draw_apple()

        for snake in snakes:
            snake.moove_snake()
            snake.draw_snake()
            if snake.detect_apple_colision(apple):
                apple = Apple(playground)
                bleService.update_apple_position(apple.coordinate)

            snake.detect_game_over()

            if (snake == mySnake):
                draw_score(snake.get_score())
        


        bleService.update_snake1_tails(snake.length, mySnake.tails)
        bleService.update_snake2_tails(snake.length, snakes[1].tails)

        display.show()
        time.sleep(0.1)
        # print(snake2.playground)
        # print(snake.playground)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

.............................